# PosgreSQL Connection

### Libraries Used

In [1]:
import psycopg2
import pandas as pd

### Connection Parameters

In [2]:
HOST = "localhost"
DATABASE = "postgres"
USER = "postgres"
PASSWORD = "rishabh"

### Connect to DataBase

In [3]:
# connect() ---> creates a new database session and returns a new instance of the connection class
conn = psycopg2.connect(host=HOST,database=DATABASE, user=USER, password=PASSWORD)

# Open a cursor to perform database operations
db_cursor = conn.cursor()

# Close communication with the database
# db_cursor.close()

### Get the list of Tables

In [4]:
# Execute a command: this creates a new table
db_cursor.execute(
    "SELECT table_schema,table_name FROM information_schema.tables WHERE table_schema = 'public';")

# retrieve data from the database 
list_tables = db_cursor.fetchall()
list_tables

[('public', 'actor'),
 ('public', 'actor_info'),
 ('public', 'customer_list'),
 ('public', 'film_list'),
 ('public', 'nicer_but_slower_film_list'),
 ('public', 'sales_by_film_category'),
 ('public', 'store'),
 ('public', 'sales_by_store'),
 ('public', 'staff_list'),
 ('public', 'address'),
 ('public', 'category'),
 ('public', 'city'),
 ('public', 'country'),
 ('public', 'customer'),
 ('public', 'film_actor'),
 ('public', 'film_category'),
 ('public', 'inventory'),
 ('public', 'language'),
 ('public', 'rental'),
 ('public', 'staff'),
 ('public', 'payment'),
 ('public', 'film')]

In [5]:
list_tables[0][1]

'actor'

### Get the Table

In [6]:
# table = db_cursor.execute("select * from actor")
table = "select * from " + list_tables[0][1]

# Read SQL query or database table into a DataFrame
df = pd.read_sql(table,conn)

### Write it to EXCEL

In [7]:
filename = list_tables[0][1] + '.xlsx' 
df.to_excel(filename,header=True)

# Mail Connection

### Libraries Used

In [8]:
import email, smtplib, ssl

from email import encoders

# This is the base class for all the MIME-specific subclasses of Message
from email.mime.base import MIMEBase

# A subclass of MIMEBase
# It is an intermediate base class for MIME messages that are multipart
from email.mime.multipart import MIMEMultipart

# MIMEText is used to create MIME objects of major_type : text
from email.mime.text import MIMEText

port = 465  # For SSL

### Connection Parameters

In [9]:
sender_email = "test.mail.bluepi@gmail.com"
receiver_email = "test.mail.bluepi@gmail.com"

subject = "An email with attachment from Python"
body = "This is an email with attachment sent from Python"

password = "rishabh_bluepi"
# input("Type your password and press enter: ")

### Send the Email

In [13]:
# Create a multipart message and set headers
message = MIMEMultipart()
message["From"] = sender_email
message["To"] = receiver_email
message["Subject"] = subject
# message["Bcc"] = receiver_email

# Add body to email
message.attach(MIMEText(body, "plain"))

# filename = list_tables[0][1] + '.xlsx'  # In same directory as script

# Open CSV file in binary mode
with open(filename, "rb") as attachment:
# Add file as application/octet-stream
# Email client can usually download this automatically as attachment
    part = MIMEBase("application", "octet-stream")
    part.set_payload(attachment.read())

# Encode file in ASCII characters to send by email    
encoders.encode_base64(part)

# Add header as key/value pair to attachment part
part.add_header(
    "Content-Disposition",
    f"attachment; filename= {filename}",
)

# Add attachment to message and convert message to string
message.attach(part)
text = message.as_string()x

# Create a secure SSL context
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, text)
